In [1]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [2]:
# ! pip install pandas numpy scikit-learn plotly matplotlib
# ! pip install nbformat
# ! pip install --upgrade nbformat
# ! pip install tensorflow[and-cuda]

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime as dt
import glob
from PIL import Image

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from tensorflow.keras.layers import Dense, Input, Dropout, Rescaling, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Precision, Recall

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings 
warnings.filterwarnings('ignore')

2024-11-06 11:13:34.247211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 11:13:34.258145: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 11:13:34.261391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 11:13:34.270588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 11:13:35.076127: W tensorflow/compiler/tf2

In [4]:
# Check if GPUs are available for training 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1730888016.568189    4476 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730888016.603792    4476 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730888016.604136    4476 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [5]:
BATCH_SIZE=32

#### Train/test sets

In [6]:
# X_train, y_train
train_imgs = glob.glob(f'../CIFAKE/train/*/*')
X_train = []
y_train = []

for path, img_path in enumerate(train_imgs):
    class_ = img_path.split('/')[-2]

    # FAKE : 0 -  REAL : 1
    if class_=='FAKE': img_class = 0
    if class_=='REAL': img_class = 1

    # X_train to tensor
    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_train.append(img_np)
    y_train.append(img_class)

X_train = np.asarray(X_train, dtype='uint8')
y_train = np.asarray(y_train)

# X_test, y_test
test_imgs = glob.glob(f'../CIFAKE/test/*/*')

X_test = []
y_test = []

for path, img_path in enumerate(test_imgs):
    class_ = img_path.split('/')[-2]

    # FAKE : 0 -  REAL : 1
    if class_=='FAKE': img_class = 0
    if class_=='REAL': img_class = 1

    # X_test to tensor
    img_pil = Image.open(img_path)
    img_np = np.asarray(img_pil)

    X_test.append(img_np)
    y_test.append(img_class)
    
X_test = np.asarray(X_train, dtype='uint8')
y_test = np.asarray(y_train)

In [7]:
print(f'X_train shape: {X_train.shape} | y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape} | y_test shape: {y_test.shape}')

X_train shape: (100000, 32, 32, 3) | y_train shape: (100000,)
X_test shape: (100000, 32, 32, 3) | y_test shape: (100000,)


#### Class balance

In [8]:
import glob
print('Class distribution: ')
print( f'Train REAL images: {len(glob.glob('../CIFAKE/train/REAL/*'))}'  )
print( f'Train FAKE images: {len(glob.glob('../CIFAKE/train/FAKE/*'))}'  )

print( f'Test REAL images: {len(glob.glob('../CIFAKE/test/REAL/*'))}'  )
print( f'Test FAKE images: {len(glob.glob('../CIFAKE/test/FAKE/*'))}'  )

Class distribution: 
Train REAL images: 50000
Train FAKE images: 50000
Test REAL images: 10000
Test FAKE images: 10000


In [9]:
# Show random image

#### Image scaling

In [10]:
# Image normalisation  1/255

#### Build model

In [26]:
# Build model
def CNN_model01(n_neurons_layer01=128, activation='relu', dropout_rate=.2):
    model = Sequential([
        Rescaling(1./255),
        Conv2D(n_neurons_layer01, 3, activation=activation),
        MaxPooling2D(),
        Flatten(),

        Dense(24, activation=activation),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile( optimizer=Adam(), 
                  loss=BinaryCrossentropy(),
                  metrics = ['accuracy', Precision(), Recall()],                
                  )

    model.build(input_shape=(None, 32, 32, 3))
    return model

In [27]:
# CNN_model01().summary()

#### Train model

In [28]:
N_EPOCHS = 30
# BATCH SIZE ?? # NOTE

In [48]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from datetime import datetime as dt


def train(model):
  time_start = dt.now()
  print(f'Start training, time: {time_start.time()}')

  history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=N_EPOCHS,
    verbose=1
  )

  print(f'Time elapsed: {dt.now() - time_start}')

  return history

# Train with GRID search
def train_gridSearch(model):

  keras_classifier = KerasClassifier(model=model)

  param_grid = dict(
      epochs=[5],
      batch_size= [64],  #[32, 64],
      # model__n_neurons_layer01 = np.arange(512,1025,256),
      model__dropout_rate = [0.2] # np.arange(.2, .51, .1),
      # model__activation = ['relu', 'tanh'],
  )

  grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid,
                      # cv=3,
                       scoring='accuracy')

  start_t = dt.now()
  print(f'GridSearch start time: {start_t.hour}:{start_t.minute:02}:{start_t.second:02}')
  grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test))

  print(f'GridSearch complete. Time elapsed: {dt.now()-start_t}')

  # Best results 
  print(f'Best parameters: {grid_result.best_params_}')
  print(f'Best result (R2): {grid_result.best_score_}')

  history = grid_result.best_estimator_.history_

  return [history, grid_result.best_estimator_]

In [49]:
history, best_model = train_gridSearch(CNN_model01)

GridSearch start time: 12:52:51
Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.7945 - loss: 0.4314 - precision_38: 0.7903 - recall_38: 0.7991 - val_accuracy: 0.8594 - val_loss: 0.3224 - val_precision_38: 0.8042 - val_recall_38: 0.9503
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.8861 - loss: 0.2815 - precision_38: 0.8814 - recall_38: 0.8930 - val_accuracy: 0.9125 - val_loss: 0.2221 - val_precision_38: 0.9096 - val_recall_38: 0.9159
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9044 - loss: 0.2391 - precision_38: 0.9043 - recall_38: 0.9034 - val_accuracy: 0.9264 - val_loss: 0.1957 - val_precision_38: 0.9191 - val_recall_38: 0.9351
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9142 - loss: 0.2161 - precision_38: 0.9159 - recall_38: 0.9129 - val_accuracy: 0.9203 - val_loss: 0.2018 - val_precision_38: 0.9564 - val_recall_38: 0.8808
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9203 - l

#### Visualisation

In [62]:
hist = pd.DataFrame(history)
cols = hist.columns

n = int(cols[-1].split('_')[-1])


dict_ = { 'loss' : hist['loss'],
         'accuracy' : hist[f'accuracy'],
         'precision' : hist[f'precision_{n}'],
         'recall' : hist[f'recall_{n}'],
         'val_loss' : hist['val_loss'],
         'val_accuracy' : hist[f'val_accuracy'],
         'val_precision' : hist[f'val_precision_{n}'],
         'val_recall' : hist[f'val_recall_{n}']

}
hist = pd.DataFrame(dict_)

In [92]:
fig_metrics = make_subplots(rows=2, cols=2, subplot_titles=("Loss", f"Accuracy", "Recall", "Precision"), vertical_spacing=0.07)

fig_metrics.add_trace( go.Scatter(x=list(range(len(hist['loss']))), y=hist['loss'], mode='lines+markers', name='Training Loss'), row=1, col=1 )
fig_metrics.add_trace( go.Scatter(x=list(range(len(hist['val_loss']))), y=hist['val_loss'], mode='lines+markers', name='Validation Loss'), row=1, col=1 )

fig_metrics.add_trace(  go.Scatter(x=list(range(len(hist['precision']))), y=hist['precision'],  mode='lines+markers', name=f'Training precision'),  row=2, col=2 )
fig_metrics.add_trace( go.Scatter(x=list(range(len(hist['val_precision']))), y=hist['val_precision'], mode='lines+markers', name=f'Validation precision'), row=2, col=2)

fig_metrics.add_trace(  go.Scatter(x=list(range(len(hist['recall']))), y=hist['recall'],  mode='lines+markers', name=f'Training recall'),  row=2, col=1 )
fig_metrics.add_trace( go.Scatter(x=list(range(len(hist['val_recall']))), y=hist['val_recall'], mode='lines+markers', name=f'Validation recall'), row=2, col=1)

fig_metrics.add_trace(  go.Scatter(x=list(range(len(hist['accuracy']))), y=hist['accuracy'],  mode='lines+markers', name=f'Training accuracy'),  row=1, col=2 )
fig_metrics.add_trace( go.Scatter(x=list(range(len(hist['val_accuracy']))), y=hist['val_accuracy'], mode='lines+markers', name=f'Validation accuracy'), row=1, col=2)


# fig.update_xaxes(title_text="Epochs", row=1, col=1)
# fig.update_yaxes(title_text="Loss", row=1, col=1)
# fig.update_yaxes(title_text=f"Accuracy", row=1, col=2)
# fig.update_yaxes(title_text=f"Recall", row=2, col=1)
# fig.update_yaxes(title_text=f"Precision", row=2, col=2)

fig_metrics.update_layout(
    # title_text="Training and validation metrics over epochs",
    showlegend=True,
    margin=dict(l=10, r=10, b=10, t=30),
    width=1400, height=800
)

for annotation in fig['layout']['annotations']:
    annotation['y'] = annotation['y'] + 0.002



# Confusion matrix
y_pred = best_model.predict(X_test)
y_pred
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
class_labels = ['FAKE', 'REAL']

# Plotly heatmap for confusion matrix
fig_confMatrix = go.Figure(data=go.Heatmap(
    z=cm,
    x= class_labels ,   # Predicted labels
    y= class_labels,    # True labels
    hoverongaps=False,
    colorscale='Blues',
    showscale=True,
    text=cm,
    texttemplate="%{text}",
    textfont={"size":15}
))

# Update layout to add labels and title
fig_confMatrix.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    width=600,
    height=500,
)




from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Plot the ROC curve using Plotly
fig_rocauc = go.Figure()

# Add the ROC curve
fig_rocauc.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    line=dict(color='blue', width=2),
    name=f'ROC curve (AUC = {roc_auc:0.2f})'
))

# Add the diagonal line (random classifier)
fig_rocauc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip'
))

# Update layout with axis titles and legend
fig_rocauc.update_layout(
    title='ROC AUC for Binary Classification',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=700,
    height=600,
    legend=dict(x=0.6, y=0.1),
    margin=dict(l=40, r=40, t=40, b=40),)




from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=class_labels, 
                               zero_division=False,
                               labels = [0, 1])

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 976us/step


In [95]:
fig = make_subplots(
    rows=4, cols=2, 
    subplot_titles=("", "",
                    "Loss", "Precision", 
                    "Accuracy", 'Recall',
                    'Confusion Matrix', 'ROC-AUC curve'), 
    horizontal_spacing=0.05, 
    vertical_spacing=0.05  
)

fig.add_trace(
    go.Scatter(
        x=[0.5], y=[0.5], 
        text=[
            # top_layers_text
        ],
        mode='text',
        showlegend=False,
    ),
    row=1, col=1  
)

fig.add_trace(
    go.Scatter(
        x=[0.5], y=[0.5], 
        text=[
            # f"Test loss: {test_loss:.4f}<br>"
            # f"Test accuracy: {test_acc:.4f}<br>"
            # f"Test precision: {test_prec:.4f}<br>"
            # f"Test recall: {test_recall:.4f}<br><br>"
            # f"ROC-AUC: {roc_auc:.4f}<br><br>"
            f"Classification report<br>"
            f"{report.replace('\n','<br>')}"
        ],
        mode='text',
        showlegend=False,
    ),
    row=1, col=2  
)

# fig.update_layout(
#     annotations=[dict(text="Model Metrics", x=0.25, y=1.05, showarrow=False, xref="paper", yref="paper", font=dict(size=16))], 
# )

# Hide the axes for the annotation row
fig.update_xaxes(visible=False, row=1, col=1)
fig.update_yaxes(visible=False, row=1, col=1)


# Loss and Precision
fig.add_trace(fig_metrics['data'][0], row=2, col=1)
fig.add_trace(fig_metrics['data'][1], row=2, col=1)
fig.add_trace(fig_metrics['data'][2], row=2, col=2)
fig.add_trace(fig_metrics['data'][3], row=2, col=2)

# Accuracy and Recall 
fig.add_trace(fig_metrics['data'][4], row=3, col=1)
fig.add_trace(fig_metrics['data'][5], row=3, col=1)
fig.add_trace(fig_metrics['data'][6], row=3, col=2)
fig.add_trace(fig_metrics['data'][7], row=3, col=2)

# Confusion Matrix and ROC-AUC curve 
fig.add_trace(fig_confMatrix['data'][0], row=4, col=1)
fig.add_trace(fig_rocauc['data'][0], row=4, col=2)
fig.add_trace(fig_rocauc['data'][1], row=4, col=2)

fig.update_layout(
    height=400*4, 
    width=1400, 
    # title_text=f"--- {MODEL.name} ---",
    showlegend=False, 
    margin=dict(l=10, r=10, t=50, b=10),  
)

fig.show()

#### Save model

In [99]:
model_name = 'GridSearch_VGG16'
best_model.model().save(f'{model_name}.h5')
best_model.model().save(f'{model_name}.keras')